In [1]:
# To run code have to run this first
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ╭────────────── Cell 1 : Core PyTorch stack (cu121) ─────────────╮
#  ▸ installs matching torch/torchvision/torchaudio wheels
!pip install --quiet --upgrade \
  torch==2.2.2+cu121 torchvision==0.17.2+cu121 torchaudio==2.2.2+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

# ╭────────────── Cell 2 : everything else we really need ─────────╮
!pip install --quiet --upgrade \
  statsmodels==0.14.1 \
  pmdarima==2.0.4 \
  properscoring==0.1 \
  optuna==3.6.1 \
  tqdm==4.67.1 \
  scikit-learn==1.5.0

# Baseline Models: Global ARIMA (for reference) and LSTM to compare with BARNN

In [3]:
# baseline_models.py ----------------------------------------------------------
# Baselines for "Novel Applications of BARNN in Retail Sales"
#   * Global ARIMA(1,1,1)
#   * Global LSTM (Optuna‑tuned, MC‑Dropout)
#   * Metrics: RMSLE / RMSE / MAE / CRPS / 90 % Coverage
# ----------------------------------------------------------------– 2025‑05‑07

# 0. LIBRARIES & GLOBALS
import os, gc, math, random, warnings, time
import numpy as np, pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from properscoring import crps_ensemble
from sklearn.metrics import mean_absolute_error, mean_squared_error

import optuna, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore", category=FutureWarning)
torch.backends.cudnn.benchmark = True

SEED         = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DATA_DIR     = "/content/drive/MyDrive/CSCI 5527 Project"   # adjust if needed
TRAIN_CUTOFF = 31              # months 0‑30 train, 31‑33 test
SEQ_LEN      = 12
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
N_MC         = 5              # MC‑Dropout samples

# ---------------------------------------------------------------------------
print("[Data] loading …")
sales = pd.read_csv(f"{DATA_DIR}/sales_train.csv")

# monthly aggregation
monthly = (sales.groupby(["date_block_num", "shop_id", "item_id"], as_index=False)
                 .agg(item_cnt_month=("item_cnt_day", "sum")))

# dense 34‑month grid
grid = (monthly[["shop_id", "item_id"]].drop_duplicates().assign(key=1)
          .merge(pd.DataFrame({"date_block_num": range(34), "key":1}), on="key")
          .drop("key", axis=1))
monthly = (grid.merge(monthly, how="left")
                .fillna({"item_cnt_month": 0})
                .sort_values(["shop_id", "item_id", "date_block_num"]))

# --- map raw IDs → contiguous indices for Embedding -------------------------
shop2idx = {sid: i for i, sid in enumerate(sorted(monthly.shop_id.unique()))}
item2idx = {iid: i for i, iid in enumerate(sorted(monthly.item_id.unique()))}

monthly["shop_idx"] = monthly.shop_id.map(shop2idx)
monthly["item_idx"] = monthly.item_id.map(item2idx)

N_SHOPS = len(shop2idx)
N_ITEMS = len(item2idx)

# ---------------------------------------------------------------------------
# 1. METRICS
def _clip(x):               # ensure non‑negative for zero‑inflated RMSLE
    return np.maximum(0, np.asarray(x))

def rmsle(y_t, y_p): return np.sqrt(np.mean((np.log1p(_clip(y_p)) -
                                             np.log1p(_clip(y_t)))**2))
def rmse(y_t, y_p): return np.sqrt(mean_squared_error(y_t, _clip(y_p)))
def mae (y_t, y_p): return mean_absolute_error(y_t, _clip(y_p))
def coverage(y_t, lo, hi):  return ((_clip(y_t) >= lo) & (_clip(y_t) <= hi)).mean()

# ---------------------------------------------------------------------------
# 2. GLOBAL ARIMA(1,1,1)
print("\n[Model] Global ARIMA(1,1,1) …")
series = (monthly.groupby("date_block_num")
                  .agg(total=("item_cnt_month", "sum"))
                  .total)
train, test = series[:TRAIN_CUTOFF], series[TRAIN_CUTOFF:]

arima = ARIMA(train, order=(1,1,1)).fit()
fc = arima.get_forecast(steps=len(test), alpha=0.10)
ar_mu = _clip(fc.predicted_mean.values)
ar_lo, ar_hi = fc.conf_int(alpha=0.10).T.values.clip(min=0)

print(f"ARIMA  | RMSLE {rmsle(test, ar_mu):.4f} | RMSE {rmse(test, ar_mu):.1f}"
      f" | MAE {mae(test, ar_mu):.1f} | Cov90 {coverage(test, ar_lo, ar_hi):.3f}"
      f" | CRPS {crps_ensemble(test.values, np.stack([ar_mu]*N_MC, 1)).mean():.3f}")

# ---------------------------------------------------------------------------
# 3. DATASET & MODEL for GLOBAL LSTM
class SalesDataset(Dataset):
    def __init__(self, frame, seq_len, mode):
        self.seq_len, self.mode = seq_len, mode
        # dict: (shop_idx, item_idx) -> 34‑length vector
        self.series = {k: g.sort_values("date_block_num").item_cnt_month.values
                       for k, g in frame.groupby(["shop_idx", "item_idx"])}
        self.keys = list(self.series)

    def __len__(self): return len(self.keys)

    def __getitem__(self, idx):
        s, i = self.keys[idx]
        vec  = self.series[(s, i)]
        if self.mode == "train":
            t = np.random.randint(self.seq_len, TRAIN_CUTOFF)
            x, y = vec[t - self.seq_len: t], vec[t]
        else:                                # deterministic slice m30→m31
            x = vec[TRAIN_CUTOFF - self.seq_len: TRAIN_CUTOFF]
            y = vec[TRAIN_CUTOFF: TRAIN_CUTOFF + 3]          # not used here
        return (torch.tensor(s, dtype=torch.long),
                torch.tensor(i, dtype=torch.long),
                torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32))

class GlobalLSTM(nn.Module):
    def __init__(self, n_shops, n_items, emb_dim, hidden, p_drop):
        super().__init__()
        self.shop_emb = nn.Embedding(n_shops, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.lstm     = nn.LSTM(1 + 2*emb_dim, hidden, batch_first=True)
        self.drop     = nn.Dropout(p_drop)
        self.head     = nn.Linear(hidden, 1)

    def forward(self, shop, item, x):        # x: (B, T)
        b, t = x.shape
        emb  = torch.cat([self.shop_emb(shop), self.item_emb(item)], -1)  # (B, 2*emb)
        emb  = emb.unsqueeze(1).repeat(1, t, 1)
        inp  = torch.cat([x.unsqueeze(-1), emb], -1)                      # (B, T, 1+2*emb)
        out, _ = self.lstm(inp)
        out   = self.drop(out[:, -1, :])
        return self.head(out).squeeze(1)

train_ds = SalesDataset(monthly, SEQ_LEN, "train")

# ---------------------------------------------------------------------------
# 4. OPTUNA TUNING (8 trials × 5 epochs)
def objective(trial):
    emb   = trial.suggest_int("emb", 16, 48, step=16)
    hid   = trial.suggest_int("hid", 32, 128, step=32)
    p_drop= trial.suggest_float("p_drop", 0.1, 0.4)
    lr    = trial.suggest_loguniform("lr", 1e-4, 3e-3)

    model = GlobalLSTM(N_SHOPS, N_ITEMS, emb, hid, p_drop).to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=lr)
    lossf = nn.MSELoss()
    loader= DataLoader(train_ds, batch_size=1024, shuffle=True,
                       num_workers=2, pin_memory=True)

    model.train()
    for _ in range(5):        # 5 quick epochs
        for s,i,x,y in loader:
            s,i,x,y = s.to(DEVICE), i.to(DEVICE), x.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            loss = lossf(model(s,i,x), y)
            loss.backward(); opt.step()

    # simple validation: month 30→31
    vt, vp = [], []
    with torch.no_grad():
        for _ in range(8000):                      # subsample for speed
            s,i = random.choice(train_ds.keys)
            vec = train_ds.series[(s,i)]
            x   = torch.tensor(vec[TRAIN_CUTOFF-SEQ_LEN:TRAIN_CUTOFF],
                               dtype=torch.float32, device=DEVICE).unsqueeze(0)
            vt.append(vec[TRAIN_CUTOFF])
            vp.append(model(torch.tensor([s],device=DEVICE),
                            torch.tensor([i],device=DEVICE), x).cpu().item())
    return rmsle(vt, vp)

print("\n[Optuna] tuning global LSTM …")
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=8, timeout=800, show_progress_bar=False)
best = study.best_params
print("Best hyper‑params:", best)

# ---------------------------------------------------------------------------
# 5. TRAIN BEST LSTM a few more epochs
model = GlobalLSTM(N_SHOPS, N_ITEMS,
                   best["emb"], best["hid"], best["p_drop"]).to(DEVICE)
opt   = torch.optim.Adam(model.parameters(), lr=best["lr"])
lossf = nn.MSELoss()
loader= DataLoader(train_ds, batch_size=1024, shuffle=True,
                   num_workers=2, pin_memory=True)

print("\n[LSTM] fine‑tuning …")
for ep in range(1, 8):   # 7 epochs
    loss_sum, n = 0.0, 0
    model.train()
    for s,i,x,y in loader:
        s,i,x,y = s.to(DEVICE), i.to(DEVICE), x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        loss = lossf(model(s,i,x), y)
        loss.backward(); opt.step()
        loss_sum += loss.item() * y.size(0); n += y.size(0)
    print(f"[{ep}/7] train MSE {loss_sum/n:.3f}")

# ---------------------------------------------------------------------------
# 6. ONE‑STEP FORECAST FOR METRICS (month 31)
def mc_pred(shop, item, vec):
    x = torch.tensor(vec[TRAIN_CUTOFF-SEQ_LEN: TRAIN_CUTOFF],
                     dtype=torch.float32, device=DEVICE).unsqueeze(0)
    s = torch.tensor([shop], device=DEVICE)
    i = torch.tensor([item], device=DEVICE)
    ys = []
    model.eval()
    with torch.no_grad():
        for _ in range(N_MC):
            ys.append(model(s,i,x).cpu().item())
    ys = np.array(ys)
    return ys.mean(), np.percentile(ys, [5, 95])

true, mu, lo, hi = [], [], [], []
for (s,i), grp in monthly.groupby(["shop_idx", "item_idx"]):
    vec = grp.sort_values("date_block_num").item_cnt_month.values
    m, (l,u) = mc_pred(s,i,vec)
    true.append(vec[TRAIN_CUTOFF]); mu.append(_clip(m)); lo.append(_clip(l)); hi.append(_clip(u))

print(f"\nLSTM   | RMSLE {rmsle(true, mu):.4f} | RMSE {rmse(true, mu):.1f}"
      f" | MAE {mae(true, mu):.1f} | Cov90 {coverage(true, lo, hi):.3f}"
      f" | CRPS {crps_ensemble(np.array(true), np.stack([mu]*N_MC,1)).mean():.3f}")

# ---------------------------------------------------------------------------
# 7. SUMMARY TABLE
print("\n------------- Baseline Summary -------------")
print("Model | RMSLE |  RMSE |  MAE | Cov90 | CRPS")
print("------|-------|-------|------|-------|------")
print(f"ARIMA | {rmsle(test, ar_mu):.4f} | {rmse(test, ar_mu):7.1f} |"
      f" {mae(test, ar_mu):6.1f} | {coverage(test, ar_lo, ar_hi):.3f} |"
      f" {crps_ensemble(test.values, np.stack([ar_mu]*N_MC,1)).mean():.3f}")
print(f"LSTM  | {rmsle(true, mu):.4f} | {rmse(true, mu):7.1f} |"
      f" {mae(true, mu):6.1f} | {coverage(true, lo, hi):.3f} |"
      f" {crps_ensemble(np.array(true), np.stack([mu]*N_MC,1)).mean():.3f}")

[Data] loading …

[Model] Global ARIMA(1,1,1) …
ARIMA  | RMSLE 0.0255 | RMSE 1787.3 | MAE 1662.4 | Cov90 1.000 | CRPS 1662.384


[I 2025-05-07 19:14:25,511] A new study created in memory with name: no-name-d47c3598-6e6f-4cbe-b39b-7dfe0d5c84be



[Optuna] tuning global LSTM …


[I 2025-05-07 19:15:19,555] Trial 0 finished with value: 0.25146524357596356 and parameters: {'emb': 48, 'hid': 96, 'p_drop': 0.10292988725822107, 'lr': 0.002708020061719506}. Best is trial 0 with value: 0.25146524357596356.
[I 2025-05-07 19:16:12,450] Trial 1 finished with value: 0.2600826966679851 and parameters: {'emb': 48, 'hid': 96, 'p_drop': 0.14866514729890173, 'lr': 0.00018994160650370794}. Best is trial 0 with value: 0.25146524357596356.
[I 2025-05-07 19:17:05,162] Trial 2 finished with value: 0.26114867126268543 and parameters: {'emb': 32, 'hid': 64, 'p_drop': 0.3762732506199805, 'lr': 0.0009838766274203484}. Best is trial 0 with value: 0.25146524357596356.
[I 2025-05-07 19:17:57,928] Trial 3 finished with value: 0.2518365889296712 and parameters: {'emb': 32, 'hid': 64, 'p_drop': 0.13586092073548445, 'lr': 0.002442198479596263}. Best is trial 0 with value: 0.25146524357596356.
[I 2025-05-07 19:18:50,443] Trial 4 finished with value: 0.2358203583449319 and parameters: {'emb': 

Best hyper‑params: {'emb': 16, 'hid': 96, 'p_drop': 0.1824822277445423, 'lr': 0.0012243823179153152}

[LSTM] fine‑tuning …
[1/7] train MSE 12.139
[2/7] train MSE 6.768
[3/7] train MSE 6.688
[4/7] train MSE 7.157
[5/7] train MSE 7.690
[6/7] train MSE 4.762
[7/7] train MSE 8.294

LSTM   | RMSLE 0.2457 | RMSE 1.3 | MAE 0.2 | Cov90 0.071 | CRPS 0.230

------------- Baseline Summary -------------
Model | RMSLE |  RMSE |  MAE | Cov90 | CRPS
------|-------|-------|------|-------|------
ARIMA | 0.0255 |  1787.3 | 1662.4 | 1.000 | 1662.384
LSTM  | 0.2457 |     1.3 |    0.2 | 0.071 | 0.230


# Baseline Models: Global ARIMA; Per Series Global ARIMA and LSTM to compare with BARNN

In [5]:
# baseline_models.py ----------------------------------------------------------
# Baselines for "Novel Applications of BARNN in Retail Sales"
#   • ARIMA(1,1,1) on the AGGREGATE total
#   • ARIMA(1,1,1) on EVERY shop‑item series  (new!)
#   • Global LSTM (Optuna‑tuned, MC‑Dropout)
#   • Metrics: RMSLE / RMSE / MAE / CRPS / 90 % Coverage
# ----------------------------------------------------------------– 2025‑05‑07

# 0. LIBRARIES & GLOBALS
import os, random, warnings, time, math, gc
import numpy as np, pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from properscoring import crps_ensemble
from sklearn.metrics import mean_absolute_error, mean_squared_error

import optuna, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from joblib import Parallel, delayed

warnings.filterwarnings("ignore", category=FutureWarning)
torch.backends.cudnn.benchmark = True

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DATA_DIR     = "/content/drive/MyDrive/CSCI 5527 Project"
TRAIN_CUTOFF = 31               # months 0‑30 train; 31‑33 test
SEQ_LEN      = 12
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
N_MC         = 5                # MC‑Dropout draws

# ────────────────────────────────────────────────────────────────────────────
print("[Data] loading …")
sales  = pd.read_csv(f"{DATA_DIR}/sales_train.csv")

monthly = (sales.groupby(["date_block_num","shop_id","item_id"], as_index=False)
                 .agg(item_cnt_month=("item_cnt_day","sum")))

grid = (monthly[["shop_id","item_id"]].drop_duplicates().assign(key=1)
          .merge(pd.DataFrame({"date_block_num": range(34), "key":1}), on="key")
          .drop("key", axis=1))
monthly = (grid.merge(monthly, how="left")
                .fillna({"item_cnt_month":0})
                .sort_values(["shop_id","item_id","date_block_num"]))

shop2idx = {sid:i for i,sid in enumerate(sorted(monthly.shop_id.unique()))}
item2idx = {iid:i for i,iid in enumerate(sorted(monthly.item_id.unique()))}
monthly["shop_idx"] = monthly.shop_id.map(shop2idx)
monthly["item_idx"] = monthly.item_id.map(item2idx)

N_SHOPS, N_ITEMS = len(shop2idx), len(item2idx)

# ─── metric helpers ─────────────────────────────────────────────────────────
_clip = lambda x: np.maximum(0, np.asarray(x))
def rmsle(y_t,y_p): return np.sqrt(np.mean((np.log1p(_clip(y_p))-
                                            np.log1p(_clip(y_t)))**2))
def rmse(y_t,y_p):  return np.sqrt(mean_squared_error(y_t,_clip(y_p)))
def mae (y_t,y_p):  return mean_absolute_error(y_t,_clip(y_p))
def coverage(y_t,lo,hi): return ((_clip(y_t)>=lo)&(_clip(y_t)<=hi)).mean()

# ---------------------------------------------------------------------------
# 1. PER‑SERIES  GLOBAL  ARIMA(1,1,1)  (new section)
# ---------------------------------------------------------------------------
print("\n[Per‑series ARIMA] fitting ≈420 k series …")

def forecast_one(vec):
    try:
        m  = ARIMA(vec[:TRAIN_CUTOFF], order=(1,1,1)).fit(method="statespace",
                                                          disp=0)
        fc = m.forecast(1)[0]
    except Exception:                    # rare convergence fail
        fc = vec[TRAIN_CUTOFF-1]         # naive fallback
    return max(fc,0.0)

series_vecs = (monthly.groupby(["shop_idx","item_idx"])
                        .apply(lambda g: g.item_cnt_month.values)
                        .tolist())

ps_preds = Parallel(n_jobs=-1, verbose=0)(
              delayed(forecast_one)(v) for v in series_vecs)

ps_true = monthly.loc[monthly.date_block_num==TRAIN_CUTOFF,
                      "item_cnt_month"].values

ps_lo = _clip(ps_preds - 1.64*np.sqrt(ps_preds))
ps_hi = _clip(ps_preds + 1.64*np.sqrt(ps_preds))

print(f"ARIMA‑series | RMSLE {rmsle(ps_true, ps_preds):.4f}"
      f" | RMSE {rmse(ps_true, ps_preds):.2f}"
      f" | MAE {mae(ps_true, ps_preds):.2f}"
      f" | Cov90 {coverage(ps_true, ps_lo, ps_hi):.3f}"
      f" | CRPS {crps_ensemble(ps_true, np.stack([ps_preds]*N_MC,1)).mean():.3f}")

# ---------------------------------------------------------------------------
# 2. GLOBAL  (aggregate‑total)  ARIMA(1,1,1)
# ---------------------------------------------------------------------------
print("\n[Model] Global ARIMA(1,1,1) on TOTAL sales …")
total_series = (monthly.groupby("date_block_num")
                        .agg(total=("item_cnt_month","sum"))
                        .total)
train_tot, test_tot = total_series[:TRAIN_CUTOFF], total_series[TRAIN_CUTOFF:]

arima_tot = ARIMA(train_tot, order=(1,1,1)).fit()
fc_tot  = arima_tot.get_forecast(steps=len(test_tot), alpha=0.10)
tot_mu  = _clip(fc_tot.predicted_mean.values)
tot_lo, tot_hi = fc_tot.conf_int(alpha=0.10).T.values.clip(min=0)

print(f"ARIMA‑total  | RMSLE {rmsle(test_tot, tot_mu):.4f}"
      f" | RMSE {rmse(test_tot, tot_mu):.1f}"
      f" | MAE {mae (test_tot, tot_mu):.1f}"
      f" | Cov90 {coverage(test_tot, tot_lo, tot_hi):.3f}"
      f" | CRPS {crps_ensemble(test_tot.values, np.stack([tot_mu]*N_MC,1)).mean():.3f}")

# ---------------------------------------------------------------------------
# 3. GLOBAL LSTM with shop/item embeddings  (unchanged)
# ---------------------------------------------------------------------------
class SalesDataset(Dataset):
    def __init__(self, frame, seq_len, mode):
        self.seq_len, self.mode = seq_len, mode
        self.series = {k: g.sort_values("date_block_num").item_cnt_month.values
                       for k,g in frame.groupby(["shop_idx","item_idx"])}
        self.keys = list(self.series)
    def __len__(self): return len(self.keys)
    def __getitem__(self, idx):
        s,i = self.keys[idx]; vec=self.series[(s,i)]
        if self.mode=="train":
            t = np.random.randint(self.seq_len, TRAIN_CUTOFF)
            x,y = vec[t-self.seq_len:t], vec[t]
        else:
            x = vec[TRAIN_CUTOFF-self.seq_len:TRAIN_CUTOFF]
            y = vec[TRAIN_CUTOFF:TRAIN_CUTOFF+3]
        return (torch.tensor(s), torch.tensor(i),
                torch.tensor(x,dtype=torch.float32),
                torch.tensor(y,dtype=torch.float32))

class GlobalLSTM(nn.Module):
    def __init__(self,n_shops,n_items,emb,hid,p_drop):
        super().__init__()
        self.shop_emb = nn.Embedding(n_shops,emb)
        self.item_emb = nn.Embedding(n_items,emb)
        self.lstm     = nn.LSTM(1+2*emb,hid,batch_first=True)
        self.drop     = nn.Dropout(p_drop)
        self.head     = nn.Linear(hid,1)
    def forward(self,shop,item,x):
        b,t = x.shape
        e = torch.cat([self.shop_emb(shop), self.item_emb(item)],-1)
        e = e.unsqueeze(1).repeat(1,t,1)
        z = torch.cat([x.unsqueeze(-1), e],-1)
        h,_ = self.lstm(z)
        h = self.drop(h[:,-1,:])
        return self.head(h).squeeze(1)

train_ds = SalesDataset(monthly,SEQ_LEN,"train")

# --- Optuna hyper‑search -----------------------------------------------------
def objective(trial):
    emb   = trial.suggest_int ("emb",16,48,step=16)
    hid   = trial.suggest_int ("hid",32,128,step=32)
    p_drop= trial.suggest_float("p_drop",0.1,0.4)
    lr    = trial.suggest_loguniform("lr",1e-4,3e-3)
    mdl   = GlobalLSTM(N_SHOPS,N_ITEMS,emb,hid,p_drop).to(DEVICE)
    opt   = torch.optim.Adam(mdl.parameters(),lr=lr)
    lossf = nn.MSELoss()
    loader= DataLoader(train_ds,batch_size=1024,shuffle=True,
                       num_workers=2,pin_memory=True)
    mdl.train()
    for _ in range(5):
        for s,i,x,y in loader:
            s,i,x,y = s.to(DEVICE),i.to(DEVICE),x.to(DEVICE),y.to(DEVICE)
            opt.zero_grad(); loss=lossf(mdl(s,i,x),y); loss.backward(); opt.step()
    # quick val m30→m31
    vt,vp=[],[]
    with torch.no_grad():
        for _ in range(8000):
            s,i = random.choice(train_ds.keys)
            vec = train_ds.series[(s,i)]
            x = torch.tensor(vec[TRAIN_CUTOFF-SEQ_LEN:TRAIN_CUTOFF],
                             dtype=torch.float32,device=DEVICE).unsqueeze(0)
            vt.append(vec[TRAIN_CUTOFF])
            vp.append(mdl(torch.tensor([s],device=DEVICE),
                          torch.tensor([i],device=DEVICE),x).cpu().item())
    return rmsle(vt,vp)

print("\n[Optuna] tuning global LSTM …")
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=8,timeout=800,show_progress_bar=False)
best = study.best_params
print("Best hyper‑params:",best)

# --- train best config -------------------------------------------------------
lstm = GlobalLSTM(N_SHOPS,N_ITEMS,best["emb"],best["hid"],best["p_drop"]).to(DEVICE)
opt  = torch.optim.Adam(lstm.parameters(),lr=best["lr"])
lossf= nn.MSELoss()
loader=DataLoader(train_ds,batch_size=1024,shuffle=True,num_workers=2,pin_memory=True)

print("\n[LSTM] fine‑tuning …")
for ep in range(1,8):
    loss_sum,n=0,0
    lstm.train()
    for s,i,x,y in loader:
        s,i,x,y=s.to(DEVICE),i.to(DEVICE),x.to(DEVICE),y.to(DEVICE)
        opt.zero_grad(); loss=lossf(lstm(s,i,x),y); loss.backward(); opt.step()
        loss_sum+=loss.item()*y.size(0); n+=y.size(0)
    print(f"[{ep}/7] train MSE {loss_sum/n:.3f}")

# --- batched MC‑Dropout inference month‑31 ----------------------------------
print("\n[LSTM] MC‑Dropout inference …")
series_keys = list(monthly.groupby(["shop_idx","item_idx"]).groups.keys())
X_all = np.stack([monthly.loc[(monthly.shop_idx==s)&(monthly.item_idx==i),
                              "item_cnt_month"].values
                  [TRAIN_CUTOFF-SEQ_LEN:TRAIN_CUTOFF] for (s,i) in series_keys])
X_all = torch.tensor(X_all,dtype=torch.float32,device=DEVICE)
S_all = torch.tensor([k[0] for k in series_keys],dtype=torch.long,device=DEVICE)
I_all = torch.tensor([k[1] for k in series_keys],dtype=torch.long,device=DEVICE)

lstm.eval(); preds=[]
with torch.no_grad():
    for _ in range(N_MC):
        preds.append(lstm(S_all,I_all,X_all).cpu().numpy())
preds = np.stack(preds)          # (N_MC, N_series)

mu  = _clip(preds.mean(0))
lo  = _clip(np.percentile(preds,5,0))
hi  = _clip(np.percentile(preds,95,0))
lstm_true = monthly.loc[monthly.date_block_num==TRAIN_CUTOFF,
                        "item_cnt_month"].values

print(f"LSTM           | RMSLE {rmsle(lstm_true,mu):.4f}"
      f" | RMSE {rmse(lstm_true,mu):.2f}"
      f" | MAE {mae(lstm_true,mu):.2f}"
      f" | Cov90 {coverage(lstm_true,lo,hi):.3f}"
      f" | CRPS {crps_ensemble(lstm_true,np.stack([mu]*N_MC,1)).mean():.3f}")

# ---------------------------------------------------------------------------
# 4. SUMMARY TABLE
print("\n------------- Baseline Summary -------------")
print("Model              | RMSLE |  RMSE |  MAE | Cov90 |  CRPS")
print("-------------------|-------|-------|------|-------|-------")
print(f"ARIMA (total)      | {rmsle(test_tot, tot_mu):.4f} |"
      f" {rmse(test_tot, tot_mu):7.1f} | {mae(test_tot, tot_mu):6.1f} |"
      f" {coverage(test_tot, tot_lo, tot_hi):.3f} |"
      f" {crps_ensemble(test_tot.values, np.stack([tot_mu]*N_MC,1)).mean():.3f}")
print(f"ARIMA (per‑series) | {rmsle(ps_true, ps_preds):.4f} |"
      f" {rmse(ps_true, ps_preds):7.1f} | {mae(ps_true, ps_preds):6.1f} |"
      f" {coverage(ps_true, ps_lo, ps_hi):.3f} |"
      f" {crps_ensemble(ps_true, np.stack([ps_preds]*N_MC,1)).mean():.3f}")
print(f"LSTM              | {rmsle(lstm_true, mu):.4f} |"
      f" {rmse(lstm_true, mu):7.1f} | {mae(lstm_true, mu):6.1f} |"
      f" {coverage(lstm_true, lo, hi):.3f} |"
      f" {crps_ensemble(lstm_true, np.stack([mu]*N_MC,1)).mean():.3f}")

[Data] loading …

[Per‑series ARIMA] fitting ≈420 k series …


<ipython-input-5-4a719b4f074c>:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.item_cnt_month.values)


ARIMA‑series | RMSLE 0.2743 | RMSE 0.95 | MAE 0.17 | Cov90 0.947 | CRPS 0.175

[Model] Global ARIMA(1,1,1) on TOTAL sales …
ARIMA‑total  | RMSLE 0.0255 | RMSE 1787.3 | MAE 1662.4 | Cov90 1.000 | CRPS 1662.384


[I 2025-05-07 21:00:37,272] A new study created in memory with name: no-name-f8e0f615-d30d-42c3-a18f-c1bbbc8c99cc



[Optuna] tuning global LSTM …


[I 2025-05-07 21:01:32,677] Trial 0 finished with value: 0.26358592260941854 and parameters: {'emb': 32, 'hid': 32, 'p_drop': 0.36848230474234633, 'lr': 0.00022518169953072038}. Best is trial 0 with value: 0.26358592260941854.
[I 2025-05-07 21:02:28,382] Trial 1 finished with value: 0.26703782405911347 and parameters: {'emb': 32, 'hid': 32, 'p_drop': 0.3405953770231652, 'lr': 0.00012697929663038347}. Best is trial 0 with value: 0.26358592260941854.
[I 2025-05-07 21:03:24,791] Trial 2 finished with value: 0.2545548770101176 and parameters: {'emb': 32, 'hid': 64, 'p_drop': 0.13487245436940312, 'lr': 0.0001336788568795171}. Best is trial 2 with value: 0.2545548770101176.
[I 2025-05-07 21:04:21,017] Trial 3 finished with value: 0.26132730764400175 and parameters: {'emb': 32, 'hid': 128, 'p_drop': 0.19449674194885913, 'lr': 0.00024538376249076395}. Best is trial 2 with value: 0.2545548770101176.
[I 2025-05-07 21:05:16,576] Trial 4 finished with value: 0.2529667528139925 and parameters: {'em

Best hyper‑params: {'emb': 16, 'hid': 64, 'p_drop': 0.35862704860509154, 'lr': 0.0022478322078263806}

[LSTM] fine‑tuning …
[1/7] train MSE 7.878
[2/7] train MSE 9.195
[3/7] train MSE 7.060
[4/7] train MSE 11.007
[5/7] train MSE 8.140
[6/7] train MSE 9.668
[7/7] train MSE 4.886

[LSTM] MC‑Dropout inference …
LSTM           | RMSLE 0.2542 | RMSE 1.26 | MAE 0.25 | Cov90 0.027 | CRPS 0.255

------------- Baseline Summary -------------
Model              | RMSLE |  RMSE |  MAE | Cov90 |  CRPS
-------------------|-------|-------|------|-------|-------
ARIMA (total)      | 0.0255 |  1787.3 | 1662.4 | 1.000 | 1662.384
ARIMA (per‑series) | 0.2743 |     0.9 |    0.2 | 0.947 | 0.175
LSTM              | 0.2542 |     1.3 |    0.3 | 0.027 | 0.255
